In [1]:
import cPickle as pickle

In [2]:
import pandas as pd
from collections import defaultdict, Counter

In [3]:
import numpy as np

In [4]:
from tqdm import tqdm

In [5]:
data=pickle.load(open("results.pickle", 'rb'))

In [6]:
data.head(5)

,corpus_id,docid,ngram_id,count,doc_length,tfidf,tf,tfidf_var
0,0,137827,23737,107,11872,0.050491,0.009013,0.039271
1,0,137827,62,89,11872,0.014588,0.007497,0.012235
2,0,137827,488,67,11872,0.016907,0.005644,0.014718
3,0,137827,22152,53,11872,0.021899,0.004464,0.017032
4,0,137827,137671,39,11872,0.018403,0.003285,0.014314


In [7]:
ngram_list=data.ngram_id.drop_duplicates().tolist()
corpus_list=data.corpus_id.drop_duplicates().tolist()

In [8]:
corpus_docid_dict=defaultdict(list)
corpus_num_doc=defaultdict(int)

for corpus in corpus_list:
    corpus_docid_dict[corpus]=data[data['corpus_id']==corpus]['docid'].drop_duplicates().tolist()
    corpus_num_doc[corpus]=len(corpus_docid_dict[corpus])

In [9]:
corpus_ngram_dict=defaultdict(list)

for corpus in corpus_list:
    corpus_ngram_dict[corpus]=data[data['corpus_id']==corpus]['ngram_id'].drop_duplicates().tolist()

In [16]:
corpus_docid_ngram_dict=defaultdict()
for corpus in corpus_list:
    corpus_docid_ngram_dict[corpus]=defaultdict(list)
    for docid in corpus_docid_dict[corpus]:
        corpus_docid_ngram_dict[corpus][docid]=data[(data['corpus_id']==corpus)&(data['docid']==docid)]['ngram_id'].drop_duplicates().tolist()
        

In [25]:
#среднее tf по ВСЕМ документам в корпусе для каждой нграммы
corpus_ngram_avg_tf=defaultdict(list)
for ngram in tqdm(ngram_list):
    for corpus in corpus_list:
        corpus_ngram_avg_tf[ngram].append(sum(data[(data['corpus_id']==corpus)&(data['ngram_id']==ngram)]\
                                            ['tf'].tolist())/corpus_num_doc[corpus]) 

100%|█████████████████████████████████| 158202/158202 [13:01<00:00, 202.52it/s]


In [27]:
#считаем дисперсию между классами (нормированную по сумме квадратов tf)
corpus_ngram_intervar_tf=defaultdict(float)
for ngram in tqdm(ngram_list):
    corpus_ngram_intervar_tf[ngram]=(np.var(corpus_ngram_avg_tf[ngram]))/(
        reduce(lambda x,y: x+y, map(lambda x: x*x, corpus_ngram_avg_tf[ngram])))
    

100%|███████████████████████████████| 158202/158202 [00:03<00:00, 43629.89it/s]


In [39]:
#записываем tf для всех документов внутри корпуса


ngram_doc_tf=defaultdict()
for corpus in corpus_list:
    ngram_doc_tf[corpus]=defaultdict()
    for docid in tqdm(corpus_docid_dict[corpus]):
        
        
        for ngram in corpus_docid_ngram_dict[corpus][docid]:
            ngram_doc_tf[corpus][ngram]=defaultdict(float)
            
            
            ngram_doc_tf[corpus][ngram][docid]=data[(data['corpus_id']==corpus) &
                                                               ((data['docid']==docid) &
                                                               (data['ngram_id']==ngram))]['tf'].tolist()[0]

            
                
                
            
            

100%|████████████████████████████████████████| 152/152 [02:15<00:00,  2.28it/s]


In [41]:
for corpus in corpus_list:
    for docid in tqdm(corpus_docid_dict[corpus]):
        for ngram in corpus_ngram_dict[corpus]:
            a=ngram_doc_tf[corpus][ngram][docid]

100%|███████████████████████████████████████| 152/152 [00:01<00:00, 147.43it/s]


In [43]:
np.var(ngram_doc_tf[0][23737].values(), dtype=float)

2.9863792675530343e-07

In [42]:
ngram_doc_tf[0][62]

defaultdict(float,
            {217: 0.0,
             693: 0.0,
             791: 0.0,
             1272: 0.0,
             1336: 0.0,
             2100: 0.0,
             2419: 0.0,
             2689: 0.0,
             3336: 0.0,
             4790: 0.0,
             5310: 0.0,
             5372: 0.0,
             5777: 0.0,
             6022: 0.0,
             6228: 0.0,
             6578: 0.0,
             6769: 0.0,
             6840: 0.0,
             6918: 0.0,
             7182: 0.0,
             9123: 0.0,
             9144: 0.0,
             9498: 0.0,
             9549: 0.0,
             10071: 0.0,
             10511: 0.0,
             10631: 0.0,
             10744: 0.0,
             11341: 0.0,
             11352: 0.0,
             11450: 0.0,
             11567: 0.0,
             12095: 0.0,
             12247: 0.0,
             12265: 0.0,
             12908: 0.0,
             13508: 0.0,
             13534: 0.0,
             13605: 0.0,
             14446: 0.0,
        

In [45]:
corpus_ngram_intravar_tf=defaultdict()

for corpus in corpus_list:
    corpus_ngram_intravar_tf[corpus]=defaultdict(int)
    for ngram in tqdm(corpus_ngram_dict[corpus]):
        corpus_ngram_intravar_tf[corpus][ngram]=(np.var(ngram_doc_tf[corpus][ngram].values()))
    
    

100%|███████████████████████████████████| 8689/8689 [00:00<00:00, 24825.72it/s]


In [46]:
indexes=[]
tfidf_fullvar=[]
for index, row in data.iterrows():
    indexes.append(index)
    tfidf_fullvar.append(row['tfidf']*corpus_ngram_intravar_tf[row['corpus_id']][row['ngram_id']]\
                        *(1-corpus_ngram_intervar_tf[row['ngram_id']]))

In [47]:
data=pd.concat([data, pd.Series(data=tfidf_fullvar,index=indexes,name='tfidf_fullvar')],axis=1)

In [48]:
data.head(5)

,corpus_id,docid,ngram_id,count,doc_length,tfidf,tf,tfidf_var,tfidf_fullvar
0,0,137827,23737,107,11872,0.050491,0.009013,0.039271,1.172770e-08
1,0,137827,62,89,11872,0.014588,0.007497,0.012235,7.028444e-07
2,0,137827,488,67,11872,0.016907,0.005644,0.014718,9.472815e-10
3,0,137827,22152,53,11872,0.021899,0.004464,0.017032,2.374237e-11
4,0,137827,137671,39,11872,0.018403,0.003285,0.014314,5.678782e-10


In [49]:
pickle.dump(data, open("results_.pickle", 'wb'))